In [1]:
from datasets import load_dataset
from transformers import (AutoModelForCausalLM, 
                            AutoTokenizer, 
                            Trainer, 
                            TrainingArguments,
                            DataCollatorForLanguageModeling)
from datasets import Dataset
import json


c:\Users\Данил\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
dataset = load_dataset(
    "json",
    data_files="../data/conversations.jsonl",
    split="train",
    streaming=True
)

In [7]:
def tokenize(example):
    return tokenizer(
        example["conversation"],  
        padding="max_length",    
        truncation=True,
        max_length=256
    )



In [8]:
model_name = "microsoft/DialoGPT-medium"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

tokenized_dataset = dataset.map(tokenize, remove_columns=["domain", "conversation"])

In [9]:
def add_labels(example):
    example["labels"] = example["input_ids"]
    return example

tokenized_dataset = tokenized_dataset.map(add_labels)

5


In [10]:
model = AutoModelForCausalLM.from_pretrained(model_name)
model.resize_token_embeddings(len(tokenizer))

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


Embedding(50257, 1024)

In [ ]:
OUTPUT_DIR = "models/rugpt-dialogue"

BATCH_SIZE = 2
EPOCHS = 3
LR = 5e-5

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    overwrite_output_dir=True,

    max_steps=50000,     
    per_device_train_batch_size=BATCH_SIZE,
    learning_rate=LR,

    logging_steps=100,
    save_steps=1000,
    save_total_limit=2,

    fp16=True,
    report_to="none",
    remove_unused_columns=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
)

trainer.train()
print("✅ Обучение завершено!")


`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
100,2.988700
200,2.226600
300,2.107200
400,2.018500
500,1.916700
600,1.867800
700,1.840000
800,1.816800


In [ ]:
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

print("✅ Обучение завершено!")

In [ ]:

model_path = 'sberbank-ai/rugpt3medium_based_on_gpt2'

model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id

text = 'Привет. Как дела?'
inputs = tokenizer(
    text,
    return_tensors="pt",
    padding=True,
    truncation=True
)

output = model.generate(
    input_ids=inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    max_new_tokens=80,
    do_sample=True,
    top_p=0.9,
    temperature=0.8,
    pad_token_id=tokenizer.eos_token_id
)


print(tokenizer.decode(output[0], skip_special_tokens=True))




If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Привет. Как дела?................................................................................
